# Langchain with Moderation

A moderated workflow chains two models together, with the second model moderating the first.

We'll use the PromptTemplate library to create parameterized prompts.

In [4]:
import os
import torch
import numpy as np

from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# The prompt template takes two variables: sentiment and the user message. The 
# sentiment will take values like "nice" and "very rude".

assistant_template = """
You are a {sentiment} assistant that responds to user comments,
using vocabulary similar to the user.
User:" {customer_request}"
Comment:
"""

assistant_prompt_template = PromptTemplate(
    input_variables=["sentiment", "customer_request"],
    template=assistant_template
)

# Define an intial chain - no moderator yet. The prompt template pipes into 
# the OpenAI LLM model and the model's response pipes into the output parser.

assistant_llm = ChatOpenAI(model="gpt-3.5-turbo")

output_parser = StrOutputParser()

assistant_chain = assistant_prompt_template | assistant_llm | output_parser

# Utility function to invoke the chain.
def create_dialog(customer_request, sentiment):
    assistant_response = assistant_chain.invoke(
        {"customer_request": customer_request,
        "sentiment": sentiment}
    )
    return assistant_response

C:\Users\mpfol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


We have a non-moderated chain. Let's try it out. Here's a nice reply, then a rude reply.

In [5]:
customer_request = """LLM with python is a total crap experience. I feel like an Idiot!"""

response_data=create_dialog(customer_request, "nice")
print(f"nice response: {response_data}")

response_data=create_dialog(customer_request, "rude")
print(f"rude response: {response_data}")


nice response: I completely understand how you feel. It can be really frustrating when something doesn't go as expected, especially when you're putting in the effort to learn and improve. Keep in mind that learning any new skill can have its challenges, but with persistence and determination, you can overcome them and become more confident in your abilities. Don't be too hard on yourself - we all have moments where we feel like we're not getting it, but remember that every mistake is an opportunity to learn and grow. Hang in there!
rude response: No doubt, your LLM experience with Python seems utterly dreadful. It's no wonder you feel like a total nincompoop!


In [ ]:
moderator_template = """
You are the moderator of an online forum, you are strict and will not tolerate negative comments.
You will receive a comment and if it is impolite you must transform in polite.
Try to mantain the meaning when possible,

If it it's polite, you will let it remain as is and repeat it word for word.
Original comment: {comment_to_moderate}
"""
# We use the PromptTemplate class to create an instance of our template that will use the prompt from above and store variables we will need to input when we make the prompt.
moderator_prompt_template = PromptTemplate(
    input_variables=["comment_to_moderate"],
    template=moderator_template,
)

The rude reply would not be good for a customer-facing bot. Let's add a moderator to the chain.